## RAG With Multi Data Source

In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [14]:
wrapper=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=wrapper)
print(wiki.name)

wikipedia


In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever 
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x00000186D3B190A0>>

In [16]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool=create_retriever_tool(retriever,"langsmith_search","Search for Information about LangChain")


In [17]:
retrieval_tool.name

'langsmith_search'

In [18]:
##Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

In [19]:
arx_wrapper=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=250)
arx=ArxivQueryRun(api_wrapper=arx_wrapper)
print(arx.name)

arxiv


In [20]:
tools=[wiki,arx,retrieval_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\llm project\\vene\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for Information about LangChain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000186B3406EE0>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x00000186D3B190A0>>, document_prompt=PromptTemplate(input_varia

In [22]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temprature=0)

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3550: UserWarning: WARNING! temprature is not default parameter.
                temprature was transferred to model_kwargs.
                Please confirm that temprature is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
from dotenv import load_dotenv

load_dotenv()
import os

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="True"
from langchain_community.llms import Ollama

llm=Ollama(model="llama2")

In [28]:
## Prompt Template

from langchain import hub 

##get the prompt from hub

prompt=hub.pull("pisles/rag-prompt-llama-ps")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="[INST]<<SYS>> You are an assistant for analyzing transcriptions from body-worn cameras, interviews, surveillance records, and other official content. Use the retrieved context to provide analysis for officials working with federal and state agencies, such as police departments or investigators. Answer concisely within three sentences. If the answer is unclear, state that you don't know.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"))]

In [36]:
# ##Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent



In [37]:
# to run agent we need agentExceutor
from langchain.agents import AgentExecutor

agent_execute=AgentExecutor(agent=agent,tools=tools,verbose=True)


In [ ]:
agent_execute.invoke({"input":"Tell me about LangSmith" })